## Номер документа

Номер документа часто указывается после символа No или N и может состоять из цифр, букв и спецсимволов. Примеры номеров:

* 67-ФЗ,

* 324-01-ЗМО,

* 824н.

Номер документа необходимо извлечь из текста «как есть», не подвергая дополнительной обработке.
Поле number должно содержать извлеченную строку. Чтобы скомпенсировать влияние ошибок распознавания символов в исходных текстах, при сравнении предсказанного и настоящего номера проверяющей системой регистр букв не учитывается.


Для каждого типа документа свой извлекатор:

* указ - в конце документа на отдельной строке № 807, может быть и в начале № 256-уг

* постановление, распоряжение, приказ - в начале после даты и места (или перед местом) в конце строки № 688-п, №_70-пп_, 63-пр,  № 25/2-г

* закон, федеральный закон - в конце документа № 89-ОЗ

Особые случаи:

* № _ 643-уГ

* 20 ноября 2014 года

№ 596-5-ЗКО

в постановлении в конце документа

* 02 ноября 2015 г. л 23/557-П в постановлении в начале (будем выбирать в качестве номера окончание строки)

In [1]:
import re
import json

In [2]:
date_reg_exprs = [
    re.compile(r'от \d?\d [А-Яа-я]+ \d\d\d\d[^\n]+'),
    re.compile(r'\d\d\.\d\d\.\d\d\d\d[^\n]+'),
    re.compile(r'\n\d?\d [А-Яа-я]+ \d\d\d\d[^\n]+')
]

date_num_reg_exprs = [
    re.compile(r'от \d?\d [А-Яа-я]+ \d\d\d\d[^\n№]+№[^\n№]+'),
    re.compile(r'\d\d\.\d\d\.\d\d\d\d[^\n№]+№[^\n№]+'),
    re.compile(r'\n\d?\d [А-Яа-я]+ \d\d\d\d[^\n№]+№[^\n№]+')
]

number_reg_exprs = [
    re.compile(r' *№°?[_ ]+([\w\-/А-Яа-я]+)[_ ]*'),
    re.compile(r'\n№°?[_ ]+([\w\-/А-Яа-я]+)\n*'),
    re.compile(r'№°?([\w\-/А-Яа-я]+)')
]

In [3]:
s = "№ 876 \n № 897"
list(map(lambda x: x.group(1), list(number_reg_exprs[0].finditer(s))))

['876', '897']

In [4]:
number_reg_exprs[0].search(s).span()[0]

0

In [5]:
from typing import List

class NumberExtractor: 
    def predict(self, test: List[str], types: List[str]) -> List[str]:
        pred_numbers = []
        for doc, doc_type in zip(test, types):
            doc = doc.lower()
            if doc_type in ['постановление', 'распоряжение', 'приказ']:
                prev_match = None
                for expr in date_num_reg_exprs:
                    match = expr.search(doc)
                    if match and prev_match and prev_match[1] > match.span()[0]:
                        prev_match = (match.group(), match.span()[0])
                if prev_match:
                    doc = prev_match[0]
            matches = []
            for expr in number_reg_exprs:
                matches += list(map(lambda x: x.group(1), list(expr.finditer(doc))))
            if matches:
                if doc_type in ['постановление', 'распоряжение', 'приказ']:
                    pred_numbers.append(matches[0])
                else:
                    pred_numbers.append(matches[-1])
            else:
                for expr in date_reg_exprs:
                    match = expr.search(doc)
                    if match:
                        matches = match.group()
                        break
                if matches:
                    pred_numbers.append(matches.strip().split(' ')[-1])
                else:
                    pred_numbers.append('')
        return pred_numbers

In [7]:
from eval_module import quality
from type_extractor import TypeExtractor
from sklearn.model_selection import train_test_split

train = []
names =[]
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        names.append(docname)
        with open(docname, 'r') as f:
            train.append((f.read(), doc_dict['label']))
# train_data, test, _, names_test = train_test_split(train, names)

test = train
test_data = [x[0] for x in test]
test_labels = [x[1] for x in test]
true_types = [x[1]['type'] for x in test]

def predict(pred_numbers):
    results = []
    for pred_number in pred_numbers:
        prediction = {"type": "",
                      "date": "",
                      "number": pred_number,
                      "authority": "",
                      "name": ""}
        results.append(prediction)
    return results

# te = TypeExtractor()
# te.train(train_data)

ne = NumberExtractor()
predicted = ne.predict(test_data, true_types)
predicted = predict(predicted)
quality(predicted, test_labels)

{'date_accuracy': 0.0,
 'number_accuracy': 0.8535973928,
 'type_f1_score': 0.0,
 'name_jaccard': 0.0,
 'authority_jaccard': 0.0,
 'subtasks_improves': 1}

In [11]:
import numpy as np

true_types = np.array(true_types)
predicted = np.array(predicted)
test_labels = np.array(test_labels)
for doc_type in ['федеральный закон', 'постановление', 'приказ', 'распоряжение','закон', 'указ']:
    ids = np.where(true_types == doc_type)
    print(f"{}{quality(predicted[ids], test_labels[ids])['number_accuracy']}")

(array([1530, 1531, 1532, ..., 3986, 3987, 3988]),)

In [10]:
y_test = [(x['number'], x['type']) for x in test_labels]
y_pred = [x['number'] for x in predicted]

i = 0
wrong_docs = []
for test, pred in zip(y_test, y_pred):
    if test[0].lower() != pred.lower():
        wrong_docs.append((names[i], test, pred))
    i += 1
    
for doc_name, test, pred in wrong_docs:
    with open(doc_name, 'r') as f:
        print(f"right value = {test}")
        print(f"wrong value = {pred}")
        print(f.read())
        print("=====")

right value = ('29-ФЗ', 'федеральный закон')
wrong value = 27
РОССИЙСКАЯ ФЕДЕРАЦИЯ
ФЕДЕРАЛЬНЫЙ ЗАКОН

О внесении изменений в отдельные законодательные акты
Российской Федерации

Принят Государственной Думой 22 февраля 2013 года
Одобрен Советом Федерации 6 марта 2013 года
Статья 1

Внести в Федеральный закон "О банках и банковской деятельностни"
(В редакции Федерального закона от 3 февраля 1996 года № 17-ФЗ)
(Ведомости Съезда народных депутатов РСФСР иВерховного Совета
РСФСР, 1990, №27, ст.357; Собрание законодательства Российской
Федерации, 1996, № 6, ст. 492; 2001, № 33, ст. 3424; 2003, № 27, ст. 2700;
№ 52, ст. 5033; 2004, № 27, ст. 2711; 2005, № 1, ст. 45; 2006, № 19, ст. 2061;
2007, № 1, ст. 9; № 31, ст. 4011; № 41, ст. 4845; 2009, № 23, ст. 2776; № 30,
ст. 3739; 2010, № 31, ст. 4193; № 47, ст. 6028; 2011, № 7, ст. 905; № 27,
ст. 3873; № 48, ст. 6730; № 50, ст. 7351; 2012, № 27, ст. 3588; № 50,

ст. 6954; № 53, ст. 7605) следующие изменения:

ЛАНОСтатья 2

В части первой статьи 52 

ПРАВИТЕЛЬСТВО ХАБАРОВСКОГО КРАЯ

ПОСТАНОВЛЕНИЕ

29.12.2014 м - 521-пр

г. Хабаровск

О внесении изменений в Порядок предоставления срочных социальных услуг
поставщиками социальных услуг в Хабаровском крае, утвержденный постановлением Правительства Хабаровского края от 31 октября 2014 г. № 406-пр

В целях совершенствования нормативных правовых актов Хабаровского края в сфере социального обслуживания граждан Правительство края
ПОСТАНОВЛЯЕТ:

1. Внести в пункт 3 приложения к Порядку предоставления срочных социальных услуг поставщиками социальных услуг в Хабаровском крае, утвержденному постановлением Правительства Хабаровского края от 31 октября
2014 г. № 406-пр, следующие изменения:

1) графу 3 изложить в следующей редакции: "предусматривает предоставление площади жилых помещений согласно нормативам, утвержденным Правительством края, в центре социальной адаптации (помощи), в том числе для
лиц без определенного места жительства и занятий, в доме-интернате (пансионате), в том числе для прес